In [1]:
import feedparser as fp
import time
from datetime import datetime, timedelta
import pytz
from bs4 import BeautifulSoup as soup
import requests
import nltk
import pandas as pd
import io
import torch.nn as nn
import numpy as np
import scipy.spatial
from dateutil.relativedelta import relativedelta
from datetime import date
from sklearn.neighbors import NearestNeighbors

In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [3]:
model = BertModel.from_pretrained('bert-base-multilingual-uncased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Li

In [4]:
def segm_id(tokenized_text):
  z=0
  segments_ids=[0,0]
  segments_ids=[]
  for a in tokenized_text:
    if a == '|':
      z=1
    segments_ids.append(z)
  #print(segments_ids)
  return segments_ids

In [5]:
def average_it(encoded_layers,length):
  sum=0
  a=1
  for layer in encoded_layers:
    #sum=sum+layer
    if(a>10):
      sum=sum+layer    
#     if(a<12):
#       sum=sum+layer
    a+=1
  sum=sum/5
  sum=sum.squeeze()
  sum=torch.sum(sum,dim=0)
  sum=sum/length
  return sum

In [6]:
def DB_article_similarity(text):
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_ids=segm_id(tokenized_text)
    segments_tensors = torch.tensor([segments_ids])
    
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')
    
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
        
    feature=average_it(encoded_layers,len(tokens_tensor))
    return feature

In [7]:
links=['http://zeenews.india.com/hindi/india.xml',
       'http://zeenews.india.com/hindi/world.xml',
       'https://www.amarujala.com/rss/breaking-news.xml',
       'http://feeds.feedburner.com/ndtvkhabar-india',
       'http://feeds.feedburner.com/ndtvkhabar-world'
       'http://feeds.feedburner.com/ndtvkhabar-latest'
       'https://navbharattimes.indiatimes.com/india/rssfeedsection/1564454.cms'
       'https://navbharattimes.indiatimes.com/world/rssfeedsection/2279801.cms']

In [8]:
db_links= ['https://www.bhaskar.com/rss-feed/1061/',
          'https://www.bhaskar.com/rss-feed/1125/']

In [10]:
# Getting the news article to form the corpus
utc = pytz.utc
df = pd.DataFrame()
posts=[]


for s in links:
    f = fp.parse(s)
    try:
         publisher = f['feed']['title']
    except KeyError:
        continue
    for e in f['entries']:
        try:
            when = e['updated_parsed']
        except KeyError:
             when = e['published_parsed']
        when =  utc.localize(datetime.fromtimestamp(time.mktime(when)))
        title = e['title']
        try:
            body = e['content']['value']
            
        except KeyError:
            body = e['summary']
        link = e['link']
        posts.append((when, publisher, title, link, body))
        posts.sort()
        posts.reverse()
        df = pd.DataFrame(posts, columns=["Time", "Publisher","Title","Link","Body"])

In [11]:
df

,Time,Publisher,Title,Link,Body
0,2020-06-26 12:21:55+00:00,Zee News Hindi: India News,"गोवा के नए डीजीपी बने IPS मुकेश कुमार मीणा, पि...",https://zeenews.india.com/hindi/india/states/g...,आखिरकार पिछले साल&nbsp;16 नवंबर से खाली पड़े डी...
1,2020-06-26 11:50:17+00:00,NDTV Khabar - India,"Coronavirus: बेंगलुरु में तेजी से बढ़े मरीज, स...",https://khabar.ndtv.com/news/india/coronavirus...,Bengaluru Coronavirus: पिछले एक हफ्ते के दौरान...
2,2020-06-26 11:47:59+00:00,NDTV Khabar - India,मानसून लगभग दो सप्ताह पहले पूरे देश में पहुंचा...,https://khabar.ndtv.com/news/india/monsoon-rea...,दक्षिण पश्चिम मानसून निर्धारित समय से दो सप्ता...
3,2020-06-26 11:45:00+00:00,Zee News Hindi: India News,कोरोना से ठीक होने वाले मरीजों की संख्या एक्टि...,https://zeenews.india.com/hindi/india/states/r...,देश भर में कोरोना के एक्टिव केसेज से ज्यादा ठी...
4,2020-06-26 11:40:09+00:00,Zee News Hindi: World News,#ZeeNewsWorldExclusive: 'तिब्‍बत में चीन ने कि...,https://zeenews.india.com/hindi/india/zeenewsw...,तिब्बत के निर्वासित राष्ट्रपति लोबसांग सांगेय ...
...,...,...,...,...,...
66,2020-06-25 10:20:32+00:00,Zee News Hindi: World News,भारत-रूस संबंधों पर ग्रहण है पाकिस्तान! लगातार...,https://zeenews.india.com/hindi/world/why-indi...,रूस (Russia) और भारत (India) के रिश्ते काफी पु...
67,2020-06-25 09:45:28+00:00,Zee News Hindi: World News,"अपने ही घर में घिरे नेपाली प्रधानमंत्री, विपक्...",https://zeenews.india.com/hindi/world/after-ch...,चीन (China) के इशारे पर भारत (India) से रिश्ते...
68,2020-06-25 07:18:21+00:00,Zee News Hindi: World News,"इस देश ने बना ली कोरोना की वैक्सीन, नाम दिया O...",https://zeenews.india.com/hindi/world/south-af...,दक्षिण अफ्रीका में कोविड-19 के इलाज के लिए एक ...
69,2020-06-25 06:33:58+00:00,Zee News Hindi: World News,भारत के इस पड़ोसी देश के हजारों लोगों ने कोरोना...,https://zeenews.india.com/hindi/world/a-countr...,म्यांमार (Myanmar) के सुदूर पश्चिम में लड़ाई क...


In [13]:
df_db = pd.DataFrame()
posts_db=[]


for s in db_links:
    f = fp.parse(s)
    try:
         publisher= f['feed']['title']
    except KeyError:
        continue
    for e in f['entries']:
        try:
            when = e['updated_parsed']
        except KeyError:
             when = e['published_parsed']
        when =  utc.localize(datetime.fromtimestamp(time.mktime(when)))
        
        title = e['title']
        try:
            body = e['content']['value']
            
        except KeyError:
            body = e['summary']
        link = e['link']
        posts_db.append((when, publisher, title, link, body))
        posts.sort()
        posts.reverse()
        df_db = pd.DataFrame(posts_db, columns=["Time", "Publisher","Title","Link","Body"])

In [16]:
df_db=df_db[:60]
df_db = df_db1.drop_duplicates(subset='Title', keep='first')
df_db

,Time,Publisher,Title,Link,Body
0,2020-06-26 12:18:13+00:00,Dainik Bhaskar,"इंटरनेशनल एयर ट्रैवल 15 जुलाई तक बंद रहेगा, पह...",https://www.bhaskar.com/national/news/internat...,"<a href=""https://www.bhaskar.com/national/news..."
22,2020-06-26 12:15:42+00:00,Dainik Bhaskar,अंतरराष्ट्रीय उड़ानों पर 15 जुलाई तक रोक जारी र...,https://www.bhaskar.com/national/news/coronavi...,"<a href=""/national/news/coronavirus-outbreak-i..."
23,2020-06-26 12:03:06+00:00,Dainik Bhaskar,सोनिया और राहुल बोले- चीन ने जमीन नहीं छीनी तो...,https://www.bhaskar.com/national/news/ladakh-g...,"<a href=""/national/news/ladakh-galwan-valley-m..."
421,2020-06-26 11:57:31+00:00,Dainik Bhaskar,रूस में 24 घंटे में संक्रमण के 6800 मामलों की ...,https://www.bhaskar.com/international/news/cor...,"<a href=""https://www.bhaskar.com/international..."
24,2020-06-26 11:29:37+00:00,Dainik Bhaskar,कोरोना संक्रमण ने संत समाज के अध्यक्ष और ऊषा म...,https://www.bhaskar.com/national/news/corona-t...,"<a href=""/national/news/corona-transition-kill..."
8,2020-06-26 10:51:23+00:00,Dainik Bhaskar,अनंतनाग के बिजबेहड़ा में आतंकियों ने जवानों पर ...,https://www.bhaskar.com/national/news/terroris...,"<a href=""https://www.bhaskar.com/national/news..."
10,2020-06-26 10:50:38+00:00,Dainik Bhaskar,प्रधानमंत्री मोदी ने रोजगार अभियान की शुरुआत क...,https://www.bhaskar.com/local/uttar-pradesh/ne...,"<a href=""https://www.bhaskar.com/local/uttar-p..."
17,2020-06-26 10:44:53+00:00,Dainik Bhaskar,असम के खेतों को सिंचाई के लिए सीमापार से नहीं ...,https://www.bhaskar.com/national/news/after-ne...,"<a href=""https://www.bhaskar.com/national/news..."
3,2020-06-26 10:40:04+00:00,Dainik Bhaskar,स्टूडेंट्स के बेस्ट परफॉर्मिंग सब्जेक्ट के नंब...,https://www.bhaskar.com/career/news/cbse-10th-...,"<a href=""https://www.bhaskar.com/career/news/c..."
28,2020-06-26 10:37:35+00:00,Dainik Bhaskar,"हरियाणा के रोहतक में भूकंप के झटके, रिक्टर पर ...",https://www.bhaskar.com/national/news/earthqua...,"<a href=""/national/news/earthquake-tremors-in-..."


In [17]:
corpus=df['Title']

In [18]:
query=df_db["Title"]
time=df_db["Time"]

In [19]:
DB_art_similarity = nn.CosineSimilarity(dim=0, eps=1e-6)
rank=1
article_vector_in_corpus=torch.zeros((len(corpus),768))
corpus_id=0
for article in corpus:
    article_vector_in_corpus[corpus_id]=DB_article_similarity(article)
    corpus_id+=1
    
corpus_id=0
article_vector_tensorc=article_vector_in_corpus.squeeze()
article_vector_tensorc

tensor([[-0.8797, -3.1189, -0.3596,  ...,  1.4598, -1.2395, -1.5461],
        [-3.5538, -5.1069,  1.4084,  ..., -3.8513, -1.4315,  0.7515],
        [-0.1623,  0.1014, -0.3293,  ..., -1.3629, -4.2871,  2.0563],
        ...,
        [ 0.7869, -0.3843,  0.4287,  ..., -1.4870, -1.4488, -2.6859],
        [ 2.1624, -3.8159, -0.7045,  ...,  1.2207, -2.1656,  3.9155],
        [-1.7150,  1.5912, -0.1707,  ..., -2.1708, -2.1900, -0.5376]])

In [20]:
article_vector_in_query=torch.zeros((len(query),768))
query_id=0
for article in query:
    article_vector_in_query[query_id]=DB_article_similarity(article)
    query_id+=1
query_id=0
article_vector_tensorq=article_vector_in_query.squeeze()
article_vector_tensorq

tensor([[-1.7088, -1.2026,  2.3193,  ..., -2.5116, -1.7263,  2.3133],
        [-3.4975, -3.8681, -1.0835,  ..., -3.1714, -7.0421,  0.0895],
        [-2.5412, -5.9145, -0.5483,  ..., -1.8711, -8.1034,  3.4469],
        ...,
        [-3.2305, -4.1351, -2.1398,  ..., -5.7291, -5.0810,  0.2764],
        [-3.8673, -4.9391, -2.0421,  ..., -3.9551, -3.6004,  1.8704],
        [-4.5170, -0.2025, -3.4963,  ..., -2.2071, -3.2232,  2.0802]])

In [21]:
knn = NearestNeighbors( metric='cosine')
knn.fit(article_vector_tensorc)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [22]:
knn.kneighbors(article_vector_tensorc);

In [23]:
input_1=pd.concat([query,time],axis=1)

In [24]:
D, N = knn.kneighbors(article_vector_tensorq, n_neighbors=2, return_distance=True)

for input_text, time,distances, neighbors in zip(input_1.Title,input_1.Time, D, N):
    print("Input text = ", input_text, "\n")
    print("Time= ", time, "\n")
    for dist, neighbor_idx in zip(distances, neighbors):
        print("idx = ", neighbor_idx)
        print(df.Title[neighbor_idx])
        print(df.Time[neighbor_idx])
        diff_time=df.Time[neighbor_idx]-time
        print("Fast_Score ={}".format(diff_time/np.timedelta64(1,'h')),"\n")

Input text =  इंटरनेशनल एयर ट्रैवल 15 जुलाई तक बंद रहेगा, पहले एविएशन मिनिस्टर ने कही थी जून-जुलाई तक शुरू करने की बात 

Time=  2020-06-26 12:18:13+00:00 

idx =  57
सोनिया ने पीएम से पूछा- जब चीन ने हमारी जमीन नहीं हड़पी तो 20 जवान कैसे हुए शहीद
2020-06-25 21:58:43+00:00
Fast_Score =-14.325 

idx =  1
Coronavirus: बेंगलुरु में तेजी से बढ़े मरीज, सीएम येदियुरप्पा ने कहा- लॉकडाउन का सवाल ही नहीं
2020-06-26 11:50:17+00:00
Fast_Score =-0.46555555555555556 

Input text =  अंतरराष्ट्रीय उड़ानों पर 15 जुलाई तक रोक जारी रहेगी, हरियाणा के गुड़गांव में शॉपिंग मॉल अगले हफ्ते खुलेंगे: देश में अब तक 4.91 लाख केस 

Time=  2020-06-26 12:15:42+00:00 

idx =  37
दिल्ली में कोरोना के करीब 74000 केस, लेकिन घबराने की बात नहीं, स्थिति अभी काबू में है: CM केजरीवाल
2020-06-26 07:50:59+00:00
Fast_Score =-4.411944444444444 

idx =  12
गुरुग्राम में अगले हफ्ते से खुलेंगे शॉपिंग मॉल, धार्मिक स्थल फिलहाल बंद ही रहेंगे
2020-06-26 10:41:24+00:00
Fast_Score =-1.5716666666666668 

Input text =  सोनिया और राहुल बोले- च

Input text =  पहले भी कई प्रोजेक्ट को लेकर उठे सवाल, मिड-डे मील का ठेका और गंगा घाटों को गोद लेने का प्रस्ताव भी खारिज हुआ 

Time=  2020-06-26 08:32:28+00:00 

idx =  3
कोरोना से ठीक होने वाले मरीजों की संख्या एक्टिव केसेज से ज्यादा, रिकवरी रेट भी बढ़ा
2020-06-26 11:45:00+00:00
Fast_Score =3.2088888888888887 

idx =  36
कोरोना: CM केजरीवाल ने कहा- दिल्ली में काबू में हैं हालात, नए मरीजों को भर्ती होने की जरूरत नहीं
2020-06-26 07:51:19+00:00
Fast_Score =-0.6858333333333333 

Input text =  पिता ने कहा- 2021 में होने वाली थी सुशांत की शादी, मुझसे आखिरी बात इसी बारे में हुई थी 

Time=  2020-06-26 08:28:52+00:00 

idx =  40
पिताजी की अंत्येष्टि में जाने के बजाए, यूपी के सीएम योगी जी कोरोना से जिंदगियां बचाने में लगे रहे: पीएम मोदी
2020-06-26 07:22:17+00:00
Fast_Score =-1.1097222222222223 

idx =  36
कोरोना: CM केजरीवाल ने कहा- दिल्ली में काबू में हैं हालात, नए मरीजों को भर्ती होने की जरूरत नहीं
2020-06-26 07:51:19+00:00
Fast_Score =-0.6258333333333334 

Input text =  पुलवामा के त्राल इलाके 

Fast_Score =6.0472222222222225 

idx =  3
कोरोना से ठीक होने वाले मरीजों की संख्या एक्टिव केसेज से ज्यादा, रिकवरी रेट भी बढ़ा
2020-06-26 11:45:00+00:00
Fast_Score =11.1025 

Input text =  6 साल में सीमा पर पाकिस्तान से घुसपैठ की 146 कोशिशें नाकाम, सुरक्षाबलों ने ऐसा करने वाले 25 आतंकी ढेर किए 

Time=  2020-06-26 00:30:56+00:00 

idx =  23
सोनिया गांधी ने फिर बोला केंद्र पर हमला, पूछा- जब चीनी घुसपैठ नहीं हुई तो 20 जवानों की जान कैसे और क्यों गई?
2020-06-26 09:37:01+00:00
Fast_Score =9.10138888888889 

idx =  57
सोनिया ने पीएम से पूछा- जब चीन ने हमारी जमीन नहीं हड़पी तो 20 जवान कैसे हुए शहीद
2020-06-25 21:58:43+00:00
Fast_Score =-2.5369444444444444 

Input text =  हर साल 1 अरब से ज्यादा लोग जानवरों की वजह से बीमार पड़ते हैं; पूरी दुनिया वेज डाइट ले, तो 30 साल में 2500 लाख करोड़ का फायदा होगा 

Time=  2020-06-26 00:20:27+00:00 

idx =  57
सोनिया ने पीएम से पूछा- जब चीन ने हमारी जमीन नहीं हड़पी तो 20 जवान कैसे हुए शहीद
2020-06-25 21:58:43+00:00
Fast_Score =-2.362222222222222 

idx =  23
स

In [28]:
DB_Fast_Score=np.sum(diff_time)
DB_Fast_Score

Timedelta('0 days 07:07:33')